# Assignment 7: Backpropagation
Read the Rojas book (https://page.mi.fu-berlin.de/rojas/neural/neuron.pdf), chapter 7.3.3 and learn about the "matrix way" of implementing backprop. 

## Ex. 7.1 XOR
Implement a two-layer artificial neural network with two input neurons and one output neuron. Choose the number of hidden neurons to your liking and add an error "neuron" to your network. Our goal is to learn the XOR function. What does the network return for random weights of all combinations of (binary) inputs? **(RESULT)**

In [739]:
# WTF is an ERROR NEURON? :D

In [740]:
import numpy as np

# Helper functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [741]:
# Datasets
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])

y_and = [a & b for a, b in X]
y_or = [a | b for a, b in X]
y_xor = [a ^ b for a, b in X]

In [742]:
class GradientDescent:
    def __init__(self, lr):
        self.lr = lr
        
    def __call__(self, weights, gradients):
        update = weights
        for i in range(len(weights)):
            update[i] = weights[i] - self.lr * gradients[i]   
        return update

In [805]:
class Network():
    def __init__(self, hidden, m, optimizer=GradientDescent(5.)):
        self.hidden = hidden
        self.m = m
        self.optimizer = optimizer
        
    # Exercise 7.1
    def fit_wo_backprop(self, x, y, epochs=1):
        input_dim = len(x[0])
        if len(np.unique(y)) == 2:
            self.output_dim = 1
        
        self.init_weights(input_dim, self.output_dim, self.m)
        
        for epoch in range(epochs):
            for x_i, y_i in zip(x,y):
                prediction = self.predict(x_i)
                print("Input: " + str(x_i) + " , Label: " + str(y_i) + " , Prediction: " + str(prediction))
                
                
    # Exercise 7.2
    def fit_with_backprop(self, x, y, epochs=100):
        input_dim = len(x[0])
        if len(np.unique(y)) == 2:
            self.output_dim = 1
        
        self.init_weights(input_dim, self.output_dim, self.m)
        
        for epoch in range(epochs):
            print("Epoch: " + str(epoch))
            predict_true = 0
            predict_false = 0
            
            for x_i, y_i in zip(x,y):
                outputs = self.feed_forward(x_i)
                gradients = self.backprop(outputs, y_i)
                self.W = self.optimizer(self.W, gradients)
                
                prediction = self.predict(x_i)
                if prediction == y_i:
                    predict_true += 1
                else:
                    predict_false += 1
                    
                print("Input: " + str(x_i) + ", Label: " + str(y_i) + ", Prediction: " + str(prediction))
                
            
            accuracy = predict_true/(predict_true+predict_false)   
            #print("predicttrue: " + str(predict_true) + "; predictfalse: " + str(predict_false))
            print("Accuracy after epoch {}: {}".format(epoch, accuracy))
            
    
    def init_weights(self, input_dim, output_dim, m):
        self.W = []
        prev_dim = input_dim
        hidden_layer_dims = [m, output_dim] 
        
        for layer in range(self.hidden):
            layer_dim = hidden_layer_dims[layer]
            self.W.append(self.init_layer(prev_dim, layer_dim))
            prev_dim = layer_dim
    
    def init_layer(self, input_dim, output_dim):
        return np.random.random((input_dim, output_dim))
    
    def feed_forward(self, x):
        """
        if (str(type(self.W)) != "<class 'list'>"):
            self.W[1] = np.unique(self.W[1], axis=0).T
            self.W = self.W.tolist()"""
        
        outputs = [x]
        out_last = x
        
        for W_i in self.W:
            out_last = sigmoid(out_last.dot(W_i))
            outputs.append(out_last)
                
        return outputs
    
    def backprop(self, outputs, y_i):
        gradients = []
        out_last = outputs[-1]
        out_last_prev = outputs[-2]
        
        e = out_last - y_i
        
        D = np.diag(out_last * (1 - out_last))
        delta = D.dot(e)
        
        gradient = np.outer(delta, out_last_prev).T
        gradients.append(gradient)

        for i in reversed(range(self.hidden)):
            output = outputs[i]
            D = np.diag(output * (1 - output))
            delta = D.dot(self.W[i]).dot(delta)
            gradients.append(np.outer(delta, outputs[i-1]).T)
        
        gradients.reverse()
        gradients = gradients[1:]
        
        """
        gradients = np.array(gradients)[::-1]
        gradients = gradients[1:]
        # Convert self.W into numpy array
        self.W = [a.reshape(-1, 2) for a in self.W]
        self.W = np.array(self.W)
        #print("backprop: " + str(type(self.W)))"""
        
        return gradients
    
    def predict(self, x):
        out_last = self.feed_forward(x)[-1]
        out_last[out_last > .5] = 1
        out_last[out_last <= .5] = 0
        return out_last
    

In [806]:
'param1 : number of hidden layers, param2: m -> number of nodes per hidden layer'
net = Network(2, 2, GradientDescent(.4))

print("####  Exercise 7.1  ####")
net.fit_wo_backprop(X, y_xor)

####  Exercise 7.1  ####
Input: [0 0] , Label: 0 , Prediction: [ 1.]
Input: [0 1] , Label: 1 , Prediction: [ 1.]
Input: [1 0] , Label: 1 , Prediction: [ 1.]
Input: [1 1] , Label: 0 , Prediction: [ 1.]


## Ex. 7.2 Backpropagation
Implement Backpropagation and optimize the weights of your neural network using the XOR training set: 

#### x, y

(0,0), 0 

(0,1), 1

(1,0), 1

(1,1), 0

How many training iterations do you need? Plot the network error over the number of iterations! **(RESULT)**

**We needed around 60 iteration with a standard gradient descent learning rate of 0.4.**

In [807]:
print("####  Exercise 7.2  ####")
net.fit_with_backprop(X, y_xor)

####  Exercise 7.2  ####
Epoch: 0
Input: [0 0], Label: 0, Prediction: [ 1.]
Input: [0 1], Label: 1, Prediction: [ 1.]
Input: [1 0], Label: 1, Prediction: [ 1.]
Input: [1 1], Label: 0, Prediction: [ 1.]
Accuracy after epoch 0: 0.5
Epoch: 1
Input: [0 0], Label: 0, Prediction: [ 1.]
Input: [0 1], Label: 1, Prediction: [ 1.]
Input: [1 0], Label: 1, Prediction: [ 1.]
Input: [1 1], Label: 0, Prediction: [ 1.]
Accuracy after epoch 1: 0.5
Epoch: 2
Input: [0 0], Label: 0, Prediction: [ 1.]
Input: [0 1], Label: 1, Prediction: [ 1.]
Input: [1 0], Label: 1, Prediction: [ 1.]
Input: [1 1], Label: 0, Prediction: [ 1.]
Accuracy after epoch 2: 0.5
Epoch: 3
Input: [0 0], Label: 0, Prediction: [ 1.]
Input: [0 1], Label: 1, Prediction: [ 1.]
Input: [1 0], Label: 1, Prediction: [ 1.]
Input: [1 1], Label: 0, Prediction: [ 1.]
Accuracy after epoch 3: 0.5
Epoch: 4
Input: [0 0], Label: 0, Prediction: [ 1.]
Input: [0 1], Label: 1, Prediction: [ 1.]
Input: [1 0], Label: 1, Prediction: [ 1.]
Input: [1 1], Label:

Input: [0 0], Label: 0, Prediction: [ 0.]
Input: [0 1], Label: 1, Prediction: [ 1.]
Input: [1 0], Label: 1, Prediction: [ 1.]
Input: [1 1], Label: 0, Prediction: [ 1.]
Accuracy after epoch 60: 0.75
Epoch: 61
Input: [0 0], Label: 0, Prediction: [ 0.]
Input: [0 1], Label: 1, Prediction: [ 1.]
Input: [1 0], Label: 1, Prediction: [ 1.]
Input: [1 1], Label: 0, Prediction: [ 1.]
Accuracy after epoch 61: 0.75
Epoch: 62
Input: [0 0], Label: 0, Prediction: [ 0.]
Input: [0 1], Label: 1, Prediction: [ 1.]
Input: [1 0], Label: 1, Prediction: [ 1.]
Input: [1 1], Label: 0, Prediction: [ 1.]
Accuracy after epoch 62: 0.75
Epoch: 63
Input: [0 0], Label: 0, Prediction: [ 0.]
Input: [0 1], Label: 1, Prediction: [ 1.]
Input: [1 0], Label: 1, Prediction: [ 1.]
Input: [1 1], Label: 0, Prediction: [ 1.]
Accuracy after epoch 63: 0.75
Epoch: 64
Input: [0 0], Label: 0, Prediction: [ 0.]
Input: [0 1], Label: 1, Prediction: [ 1.]
Input: [1 0], Label: 1, Prediction: [ 1.]
Input: [1 1], Label: 0, Prediction: [ 0.]


# Ex. 7.3 MNIST (BONUS)
Train your network on the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) and state the model accuracy (or the model error) for the training and test sets. **(RESULT)** Compare to this [list](https://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#4d4e495354)
